In [1]:
# 🍽️ Food Image Classification using Popular Pretrained Models (Keras)

# ✅ 1. Install Required Packages
#!pip install -q tensorflow keras gdown

import os
import zipfile
import gdown
import numpy as np
import pandas as pd
from PIL import Image, UnidentifiedImageError

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.mixed_precision import set_global_policy

# ⚙️ 2. Enable Mixed Precision (for performance boost)
set_global_policy('mixed_float16')

# 📥 3. Download and Extract Dataset
OUTPUT_ZIP = 'food_data.zip'
EXTRACT_DIR = 'data'

if not os.path.exists(EXTRACT_DIR):
  DATA_URL = 'https://drive.google.com/uc?id=15CHt2ueS4c7emHpmzFHC3c0TGd51Mnvz'
  gdown.download(DATA_URL, OUTPUT_ZIP, quiet=False)
  with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zip_ref:
      zip_ref.extractall(EXTRACT_DIR)

# 🧹 4. Clean corrupted/non-image files
def clean_directory(directory):
    for root, _, files in os.walk(directory):
        for fname in files:
            fpath = os.path.join(root, fname)
            try:
                with Image.open(fpath) as img:
                    img.verify()
            except (UnidentifiedImageError, OSError):
                print(f"Removing: {fpath}")
                os.remove(fpath)

clean_directory('data/train')
clean_directory('data/test')

# 📁 5. Data Preparation
IMG_SIZE = (224, 224)
BATCH_SIZE = 64
TRAIN_DIR = 'data/train'
TEST_DIR = 'data/test'

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)

test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_data = train_gen.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE, batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

num_classes = len(train_data.class_indices)
class_indices = train_data.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# 🧠 6. Define Function to Build Transfer Learning Models

def build_transfer_model(base_model_fn, model_name):
    print(f"\n📦 Building model: {model_name}")
    base = base_model_fn(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base.trainable = False  # Freeze all pretrained layers

    # Custom layers: one at the beginning and one at the end
    model = keras.Sequential([
        layers.Input(shape=(224, 224, 3)),                      # Custom input layer
        layers.Conv2D(3, (1, 1), padding='same'),              # Layer at the beginning
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),                  # Layer at the end
        layers.Dense(num_classes, activation='softmax', dtype='float32')
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# 🔁 7. Train Multiple Models
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2, ResNet50, VGG16, InceptionV3

models_to_train = [
    (EfficientNetB0, 'EfficientNetB0'),
    (MobileNetV2, 'MobileNetV2'),
    (ResNet50, 'ResNet50'),
    (VGG16, 'VGG16'),
    (InceptionV3, 'InceptionV3')
]

for base_fn, name in models_to_train:
    try:
        model = build_transfer_model(base_fn, name)
        callbacks = [
            EarlyStopping(patience=3, restore_best_weights=True),
            ModelCheckpoint(f'{name}.h5', save_best_only=True)
        ]
        model.fit(
            train_data,
            epochs=10,
            validation_data=val_data,
            callbacks=callbacks
        )
    except Exception as e:
        print(f"⚠️ Skipped {name} due to: {e}")


Downloading...
From (original): https://drive.google.com/uc?id=15CHt2ueS4c7emHpmzFHC3c0TGd51Mnvz
From (redirected): https://drive.google.com/uc?id=15CHt2ueS4c7emHpmzFHC3c0TGd51Mnvz&confirm=t&uuid=c4b716ad-4efb-4afb-94b8-a4a138f8b9dc
To: /content/food_data.zip
100%|██████████| 898M/898M [00:21<00:00, 41.8MB/s]


Removing: data/train/chicken/6fcf7983-225.jpg
Removing: data/train/chicken/1b4ed8be-285.jpg
Removing: data/train/chicken/e5574159-a60.jpg
Removing: data/train/chicken/052f1133-141.jpg
Removing: data/train/chicken/2a2a3166-2d2.jpg
Found 19276 images belonging to 22 classes.
Found 4808 images belonging to 22 classes.

📦 Building model: EfficientNetB0
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 900ms/step - accuracy: 0.0849 - loss: 2.9947

302/302 ━━━━━━━━━━━━━━━━━━━━ 387s 1s/step - accuracy: 0.0849 - loss: 2.9946 - val_accuracy: 0.0996 - val_loss: 2.9499
Epoch 2/10
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 847ms/step - accuracy: 0.0966 - loss: 2.9527

...

302/302 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.5449 - loss: 1.5440 - val_accuracy: 0.5351 - val_loss: 1.5638
Epoch 7/10
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 810ms/step - accuracy: 0.5633 - loss: 1.4720

302/302 ━━━━━━━━━━━━━━━━━━━━ 305s 1s/step - accuracy: 0.5633 - loss: 1.4720 - val_accuracy: 0.5420 - val_loss: 1.5475
Epoch 8/10
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 808ms/step - accuracy: 0.5744 - loss: 1.4317

302/302 ━━━━━━━━━━━━━━━━━━━━ 305s 1s/step - accuracy: 0.5744 - loss: 1.4317 - val_accuracy: 0.5514 - val_loss: 1.5235
Epoch 9/10
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 812ms/step - accuracy: 0.5847 - loss: 1.3987

302/302 ━━━━━━━━━━━━━━━━━━━━ 306s 1s/step - accuracy: 0.5847 - loss: 1.3988 - val_accuracy: 0.5618 - val_loss: 1.4809
Epoch 10/10
302/302 ━━━━━━━━━━━━━━━━━━━━ 0s 819ms/step - accuracy: 0.5878 - loss: 1.3757

302/302 ━━━━━━━━━━━━━━━━━━━━ 308s 1s/step - accuracy: 0.5878 - loss: 1.3757 - val_accuracy: 0.5611 - val_loss: 1.4726
